In [45]:
import pandas as pd
import requests
import math
from decimal import Decimal

In [46]:
symbol = "SOLUSDT"
url = "https://api.binance.com/api/v1/depth"

interval = Decimal('0.1')

params = {
    "symbol": symbol,
    "limit": 100,
}

In [47]:
# bids: [price, volume], asks: [price, volume]
data = requests.get(url, params).json()
print(data)

{'lastUpdateId': 25307587848, 'bids': [['185.50000000', '192.01400000'], ['185.49000000', '182.56700000'], ['185.48000000', '397.88700000'], ['185.47000000', '284.92900000'], ['185.46000000', '193.05100000'], ['185.45000000', '307.06100000'], ['185.44000000', '135.61700000'], ['185.43000000', '400.36700000'], ['185.42000000', '841.89800000'], ['185.41000000', '382.68000000'], ['185.40000000', '277.17400000'], ['185.39000000', '307.74300000'], ['185.38000000', '436.63700000'], ['185.37000000', '282.78900000'], ['185.36000000', '193.20100000'], ['185.35000000', '389.96100000'], ['185.34000000', '629.17000000'], ['185.33000000', '358.82000000'], ['185.32000000', '634.58000000'], ['185.31000000', '185.64000000'], ['185.30000000', '506.96200000'], ['185.29000000', '344.42400000'], ['185.28000000', '198.75100000'], ['185.27000000', '350.52400000'], ['185.26000000', '384.43400000'], ['185.25000000', '229.55600000'], ['185.24000000', '359.61600000'], ['185.23000000', '475.41100000'], ['185.220

## Aggregating Bids Levels

In [48]:
bids_levels = pd.DataFrame(data['bids'], columns=['price', 'quantity'], dtype=float)
bids_levels['side'] = 'bid'
print(bids_levels)
print(bids_levels.dtypes)

     price  quantity side
0   185.50   192.014  bid
1   185.49   182.567  bid
2   185.48   397.887  bid
3   185.47   284.929  bid
4   185.46   193.051  bid
..     ...       ...  ...
95  184.55     3.797  bid
96  184.54     0.754  bid
97  184.53     0.416  bid
98  184.52   147.467  bid
99  184.51     2.186  bid

[100 rows x 3 columns]
price       float64
quantity    float64
side         object
dtype: object


In [49]:
min_bid_level = math.floor(min(bids_levels['price']) / float(interval)) * interval
min_bid_level


Decimal('184.5')

In [50]:
max_bid_level = (math.ceil(max(bids_levels['price']) / float(interval)) + 1) * interval
max_bid_level

Decimal('185.6')

In [51]:
bid_levels_bounds = [float(min_bid_level + interval * i)
    for i in range(int((max_bid_level - min_bid_level) / interval) + 1)
]
bid_levels_bounds

[184.5,
 184.6,
 184.7,
 184.8,
 184.9,
 185.0,
 185.1,
 185.2,
 185.3,
 185.4,
 185.5,
 185.6]

In [52]:
bids_levels["bin"] = pd.cut(
    bids_levels['price'],
    bins=bid_levels_bounds,
    right=False,
    precision=10
)
bids_levels

,price,quantity,side,bin
0,185.50,192.014,bid,"[185.5, 185.6)"
1,185.49,182.567,bid,"[185.4, 185.5)"
2,185.48,397.887,bid,"[185.4, 185.5)"
3,185.47,284.929,bid,"[185.4, 185.5)"
4,185.46,193.051,bid,"[185.4, 185.5)"
...,...,...,...,...
95,184.55,3.797,bid,"[184.5, 184.6)"
96,184.54,0.754,bid,"[184.5, 184.6)"
97,184.53,0.416,bid,"[184.5, 184.6)"
98,184.52,147.467,bid,"[184.5, 184.6)"


In [53]:
bids_levels = bids_levels.groupby("bin").agg(
    quantity = ("quantity", "sum"),
    side = ("side", "first")
).reset_index()

bids_levels['label'] = bids_levels['bin'].apply(lambda x: x.left)
bids_levels

C:\Users\Saksham Maggu\AppData\Local\Temp\ipykernel_24320\1399697312.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bids_levels = bids_levels.groupby("bin").agg(


,bin,quantity,side,label
0,"[184.5, 184.6)",1428.148,bid,184.5
1,"[184.6, 184.7)",1273.424,bid,184.6
2,"[184.7, 184.8)",836.918,bid,184.7
3,"[184.8, 184.9)",2816.527,bid,184.8
4,"[184.9, 185.0)",4554.798,bid,184.9
5,"[185.0, 185.1)",4612.425,bid,185.0
6,"[185.1, 185.2)",2989.879,bid,185.1
7,"[185.2, 185.3)",3198.230,bid,185.2
8,"[185.3, 185.4)",3925.503,bid,185.3
9,"[185.4, 185.5)",3403.231,bid,185.4
